In [1]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
import pandas as pd
import geopandas as gpd
import folium
import contextily as cx

%matplotlib inline
ox.config(log_console=True, use_cache=True)
ox.__version__

C:\Users\maaik\miniconda3\envs\ox\lib\site-packages\osmnx\utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


'1.3.1.post0'

---
### Build Matrix for the Game

TO DO: Edge betweenness centrality gives slightly different values. Need to choose one of the two

2. Build a gdf that includes the data we need. So the edges and the criticality values, and ID numbers

TO DO: check the warning

In [2]:
Segougraph = ox.load_graphml('./data/Maligraph120723.graphml')


FileNotFoundError: [Errno 2] No such file or directory: 'data\\Maligraph120723.graphml'

In [ ]:
#take the edges and nodes out of the graph
gdf_nodes, gdf_edges = ox.graph_to_gdfs(Segougraph)
#reset edge index
edgecriticality = gdf_edges.reset_index()

#create ID numbers (TO DO later: make the rows and columns named like the ID numbers instead of normal numbers
edgecriticality['IDnr'] = np.nan

for i in range(0, len(edgecriticality)):
    string = 'id'
    edgecriticality.IDnr[i] = (string+str(i))

In [1062]:
edgecriticality

,u,v,key,osmid,ref,highway,oneway,reversed,length,bridge,geometry,criticality,name,lanes,junction,IDnr
0,2119463951,348228117,0,"[201944290, 203179171, 203179172, 203179170, 4...",RN6,primary,False,False,44811.471,yes,"LINESTRING (-5.75949 12.94968, -5.75933 12.949...",2.688367e-06,NaN,NaN,NaN,id0
1,2119463951,2119463921,0,201944295,RN6,primary,True,False,58.671,NaN,"LINESTRING (-5.76002 12.94955, -5.75986 12.949...",0.000000e+00,NaN,NaN,NaN,id1
2,2119463951,2119463963,0,201944296,RN6,primary,True,False,53.847,NaN,"LINESTRING (-5.75949 12.94968, -5.75969 12.949...",4.668934e-07,NaN,NaN,NaN,id2
3,9372853264,2222343932,0,"[1061150372, 1061150373, 1057803463, 105780346...",NaN,primary,True,False,2313.210,NaN,"LINESTRING (-6.26126 13.43097, -6.25969 13.431...",1.333835e-05,Avenue de l'An 2000,NaN,NaN,id3
4,9372853264,8190645651,0,"[894160190, 444929633, 230159386, 1016169342]",RN33;RN34,primary,True,False,490.313,yes,"LINESTRING (-6.24073 13.43680, -6.24032 13.436...",9.265265e-08,Avenue de l'An 2000,NaN,NaN,id4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,323195349,2109314025,0,"[29893826, 188573922, 29893828, 207766664, 207...",RN33,primary,False,True,22870.786,yes,"LINESTRING (-6.07552 13.67802, -6.07539 13.677...",7.446995e-02,NaN,NaN,NaN,id202
203,2611652056,2109314027,0,914594755,RN33,primary,True,False,14.787,NaN,"LINESTRING (-6.07577 13.67833, -6.07579 13.678...",0.000000e+00,NaN,NaN,NaN,id203
204,2611652056,2109314028,0,914594757,RN33,primary,True,False,32.201,NaN,"LINESTRING (-6.07577 13.67833, -6.07591 13.678...",0.000000e+00,NaN,NaN,NaN,id204
205,2109314027,2109314028,0,914594756,RN33,primary,True,False,32.128,NaN,"LINESTRING (-6.07606 13.67839, -6.07588 13.678...",7.483243e-07,NaN,NaN,NaN,id205


3. Build a matrix with all the edges in the rows and all edges in the columns. Positive values on the diagonal and the rest negative values.

In [1063]:
nredges = len(edgecriticality)
gamematrix = pd.DataFrame(np.zeros([nredges, nredges])*np.nan)

#make a matrix with all the roads
for i in range(0, len(edgecriticality)):
    criticalityval = edgecriticality.criticality[i]
    gamematrix.iloc[:,i] = - criticalityval
    gamematrix.iloc[i,i] = criticalityval

4. Save the matrix 

In [947]:
np.save('results/game_matrixSegou08072023.npy', gamematrix)

In [1064]:
pd.DataFrame(gamematrix).to_csv("results/game_matrixSegou100720231400.csv", header=None, index=None, sep=' ')

In [1065]:
gamematrix

,0,1,2,3,4,5,6,7,8,9,...,197,198,199,200,201,202,203,204,205,206
0,0.000003,-0.0,-4.668934e-07,-0.000013,-9.265265e-08,-0.000013,-0.000003,-0.0,-0.0,-0.0,...,-0.0,-0.000032,-0.0,-0.0,-0.000004,-0.07447,-0.0,-0.0,-7.483243e-07,-0.0
1,-0.000003,0.0,-4.668934e-07,-0.000013,-9.265265e-08,-0.000013,-0.000003,-0.0,-0.0,-0.0,...,-0.0,-0.000032,-0.0,-0.0,-0.000004,-0.07447,-0.0,-0.0,-7.483243e-07,-0.0
2,-0.000003,-0.0,4.668934e-07,-0.000013,-9.265265e-08,-0.000013,-0.000003,-0.0,-0.0,-0.0,...,-0.0,-0.000032,-0.0,-0.0,-0.000004,-0.07447,-0.0,-0.0,-7.483243e-07,-0.0
3,-0.000003,-0.0,-4.668934e-07,0.000013,-9.265265e-08,-0.000013,-0.000003,-0.0,-0.0,-0.0,...,-0.0,-0.000032,-0.0,-0.0,-0.000004,-0.07447,-0.0,-0.0,-7.483243e-07,-0.0
4,-0.000003,-0.0,-4.668934e-07,-0.000013,9.265265e-08,-0.000013,-0.000003,-0.0,-0.0,-0.0,...,-0.0,-0.000032,-0.0,-0.0,-0.000004,-0.07447,-0.0,-0.0,-7.483243e-07,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,-0.000003,-0.0,-4.668934e-07,-0.000013,-9.265265e-08,-0.000013,-0.000003,-0.0,-0.0,-0.0,...,-0.0,-0.000032,-0.0,-0.0,-0.000004,0.07447,-0.0,-0.0,-7.483243e-07,-0.0
203,-0.000003,-0.0,-4.668934e-07,-0.000013,-9.265265e-08,-0.000013,-0.000003,-0.0,-0.0,-0.0,...,-0.0,-0.000032,-0.0,-0.0,-0.000004,-0.07447,0.0,-0.0,-7.483243e-07,-0.0
204,-0.000003,-0.0,-4.668934e-07,-0.000013,-9.265265e-08,-0.000013,-0.000003,-0.0,-0.0,-0.0,...,-0.0,-0.000032,-0.0,-0.0,-0.000004,-0.07447,-0.0,0.0,-7.483243e-07,-0.0
205,-0.000003,-0.0,-4.668934e-07,-0.000013,-9.265265e-08,-0.000013,-0.000003,-0.0,-0.0,-0.0,...,-0.0,-0.000032,-0.0,-0.0,-0.000004,-0.07447,-0.0,-0.0,7.483243e-07,-0.0


In [1026]:
edgecriticality.criticality.min()

0.0